<a href="https://colab.research.google.com/github/TIANQIXU1/numpy-ml/blob/master/finetune-MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate

In [2]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.2 MB/s eta 0:00:00


In [4]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [8]:
from google.colab import files
uploaded = files.upload()


Saving combined_type_id_description.csv to combined_type_id_description.csv


In [8]:
import pandas as pd
df = pd.read_csv("combined_type_id_description.csv")  # Replace with the uploaded file's name
print(df.head())

   type   id       en_description zh_description
0  Word  w15           AFP levels          AFP水平
1  Word  w15               AND/OR            和/或
2  Word  w33       Abdominal pain            ​腹痛
3  Word   w3  Ablation Techniques            消融术
4  Word   w5             Abortion             流产


In [9]:
# Load the BLEU metric
metric = evaluate.load("sacrebleu")

# Load the uploaded dataset
dataset_path = "combined_type_id_description.csv"  # File uploaded to Colab
df = pd.read_csv(dataset_path)

# Use the top 1600 entries
df = df.head(1600)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
# Prepare the dataset in parallel format
data = {
    "source": df["en_description"].tolist(),  # English descriptions
    "target": df["zh_description"].tolist(),  # Chinese translations
}

In [16]:
# Split the dataset into training and validation sets (80%-20% split)
train_size = int(0.8 * len(data["source"]))
train_data = {
    "source": data["source"][:train_size],
    "target": data["target"][:train_size],
}
val_data = {
    "source": data["source"][train_size:],
    "target": data["target"][train_size:],
}

In [11]:
# Convert the dataset to a Hugging Face Dataset
train_dataset = Dataset.from_dict(data)


In [12]:
# Load MarianMT tokenizer and model
model_name = "Helsinki-NLP/opus-mt-en-zh"  # Pretrained English-Chinese MarianMT model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [15]:
# Tokenization function
def preprocess_function(examples):
    model_inputs = tokenizer(examples["source"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
# Tokenize the entire dataset
tokenized_train = train_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [17]:
# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [18]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./marianmt-cancer-finetuned",
    evaluation_strategy="no",  # No validation
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,  # Increase epochs since we have no validation
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:

# Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-19-5553b3fe0625>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [20]:
# Fine-tune the model
print("Starting fine-tuning...")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Step,Training Loss
100,0.465900
200,0.103500
300,0.082600
400,0.072000
500,0.066600


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.15811646175384522, metrics={'train_runtime': 11715.0548, 'train_samples_per_second': 0.683, 'train_steps_per_second': 0.043, 'total_flos': 271186919424000.0, 'train_loss': 0.15811646175384522, 'epoch': 5.0})

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
output_dir = "/content/drive/My Drive/marianmt-cancer-finetuned"

# Save the fine-tuned model
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Fine-tuned model and tokenizer saved to {output_dir}")

Fine-tuned model and tokenizer saved to /content/drive/My Drive/marianmt-cancer-finetuned


In [23]:
# Zip the model directory
!zip -r marianmt-cancer-finetuned.zip ./marianmt-cancer-finetuned

# Download the zipped model
from google.colab import files
files.download("marianmt-cancer-finetuned.zip")


  adding: marianmt-cancer-finetuned/ (stored 0%)
  adding: marianmt-cancer-finetuned/checkpoint-300/ (stored 0%)
  adding: marianmt-cancer-finetuned/checkpoint-300/training_args.bin (deflated 51%)
  adding: marianmt-cancer-finetuned/checkpoint-300/scheduler.pt (deflated 55%)
  adding: marianmt-cancer-finetuned/checkpoint-300/tokenizer_config.json (deflated 68%)
  adding: marianmt-cancer-finetuned/checkpoint-300/config.json (deflated 61%)
  adding: marianmt-cancer-finetuned/checkpoint-300/rng_state.pth (deflated 24%)
  adding: marianmt-cancer-finetuned/checkpoint-300/vocab.json (deflated 68%)
  adding: marianmt-cancer-finetuned/checkpoint-300/generation_config.json (deflated 43%)
  adding: marianmt-cancer-finetuned/checkpoint-300/source.spm (deflated 49%)
  adding: marianmt-cancer-finetuned/checkpoint-300/trainer_state.json (deflated 61%)
  adding: marianmt-cancer-finetuned/checkpoint-300/model.safetensors (deflated 7%)
  adding: marianmt-cancer-finetuned/checkpoint-300/target.spm (defl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
import os
print("Saved files:", os.listdir(output_dir))


Saved files: ['config.json', 'generation_config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.json', 'source.spm', 'target.spm']


In [25]:
# Function to compute BLEU score
def compute_bleu(model, tokenizer, dataset):
    """
    Compute BLEU score for a given model, tokenizer, and dataset.
    """
    source_texts = dataset["source"]
    target_texts = [text for text in dataset["target"]]  # Ground-truth translations

    # Generate translations
    translated_texts = []
    for text in source_texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        outputs = model.generate(**inputs)
        translated_texts.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

    # Compute BLEU score
    results = metric.compute(predictions=translated_texts, references=[[t] for t in target_texts])
    return results["score"]

# Evaluate BLEU score before fine-tuning (using the pretrained model)
print("Evaluating BLEU score before fine-tuning...")
bleu_before = compute_bleu(model, tokenizer, val_data)
print(f"BLEU score before fine-tuning: {bleu_before}")

# Evaluate BLEU score after fine-tuning
print("Evaluating BLEU score after fine-tuning...")
bleu_after = compute_bleu(model, tokenizer, val_data)
print(f"BLEU score after fine-tuning: {bleu_after}")


Evaluating BLEU score before fine-tuning...
BLEU score before fine-tuning: 6.331300653999455
Evaluating BLEU score after fine-tuning...
BLEU score after fine-tuning: 6.331300653999455
